## BATTLE OF THE NEIGHBORHOODS (Toronto)
# 1. Scraping the Wiki then converting to DataFrame

In [1]:
import json
import numpy as np
import pandas as pd
import requests
import lxml.html
from bs4 import BeautifulSoup
!conda update -n base conda --yes

## was getting "failed with initial frozen solve" error without the following...
!conda create -n opencv --yes
!conda activate opencv --yes
!conda install -c conda-forge opencv --yes
##

### sets up the environment to ensure that the Wiki can be scraped
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge html5lib --yes
!conda install -c conda-forge requests --yes
### ...since it wasn't working without it

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1h             |       he774522_0         4.8 MB
    ------------------------------------------------------------
                                           Total:         4.8 MB

The following packages will be REMOVED:

  python_abi-3.8-1_cp38

The following packages will be SUPERSEDED by a higher-priority channel:

  altair                                        conda-forge --> pkgs/main
  conda              conda-forge::conda-4.9.2-py38haa244fe~ --> pkgs/main::conda-4.9.2-py38haa95532_0
  openssl                                       conda-forge --> pkgs/main




openssl-1.1.1h       | 4.8 MB    |            |   0% 
openssl-1.1.1h       | 4.8 MB    |            |   0% 
openssl-1

WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(139): Could not remove or rename C:\ProgramData\Anaconda3\pkgs\openssl-1.1.1h-he774522_0\Library\bin\libcrypto-1_1-x64.dll.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(139): Could not remove or rename C:\ProgramData\Anaconda3\pkgs\openssl-1.1.1h-he774522_0\Library\bin\libcrypto-1_1-x64.dll.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.conda_trash.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(139): Could not remove or rename C:\ProgramData\Anaconda3\pkgs\openssl-1.

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3\envs\opencv



Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
#
# To activate this environment, use
#
#     $ conda activate opencv
#
# To deactivate an active environment, use
#
#     $ conda deactivate



activate does not accept more than one argument:
['opencv', '--yes']



Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



### Then grabbing the Wikipedia table and converting it to a DataFrame, with 180 rows. I did not need to use BeautifulSoup to scrape the table, after all.

In [14]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
## Pulls the table from the Wikipedia page, then converts it to a DataFrame
toronto_wiki = pd.read_html(url)
#unknown_df = toronto_wiki[1]
toronto_df = toronto_wiki[0]
toronto_df = toronto_df.rename(columns = {'Neighbourhood':'Neighborhood'})
toronto_df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


### ...that is, until missing Boroughs are removed and the missing Neighborhoods that are removed are named after their corresponding Boroughs.

## My resulting DF is not in the same order as what Coursera depicts but all of the same information is contained in it.

In [15]:
### Ensures we only use rows with Borough names assigned (i.e., drops 'Not assigned' rows)
numspots = list(toronto_df.index)  #OR numboroughs = np.arange(1,toronto_df[['Borough']].shape[0])
temp = pd.DataFrame(columns=toronto_df.keys())
for i in numspots:
    if toronto_df[['Borough']].iloc[i,0]!="Not assigned":
        temp.loc[len(temp.index)]   =   toronto_df.iloc[i]
        
nohood_inds = []
inds = list(toronto_df.index)
missinghoods  =  toronto_df['Neighborhood'] == "Not assigned"
for i in inds:
    if missinghoods[i]==True:
        nohood_inds.append(i)
        toronto_df['Neighborhood'][i]   =   toronto_df['Borough'][i]

toronto_df = temp
toronto_df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


#### ...which has no more "Not assigned" cells, as desired.

#### The following combines all Neighborhoods in each Postal Code into a single string under Neighbourhoods then removes all but one row with each corresponding Postal Code (i.e., beforehand, some Postal Codes may repeat themselves and "need" to be collapsed to put the focus on analyzing the Neighborhoods instead). Note that this code assumes that the Borough names were the same for any such repeated Postal Codes, hence the rows removed due to duplicate Postal Codes assume that a Postal Code would never exist across more than one Borough.  #assumed since there were no repeats

In [26]:
## makes a list of unique Neighbourhood names (i.e., removes duplicates)
def unique_hoods(bloatedlist):
    ulist = []
    [ulist.append(str(hood)) for hood in bloatedlist if hood not in ulist]
    return ulist

### this segment searches for duplicate postal codes, if any
i=0
nother=0
rpc = {}
postcodes = list(toronto_df['Postal Code'].value_counts().index) 
for code in postcodes:  #given this particular Postal Code...
    if  toronto_df['Postal Code'].value_counts().loc[[code]][0]  > 1:  #...if this Postal Code has repeats elsewhere, then
        comm_inds = list(toronto_df[ toronto_df['Postal Code']==code ].Neighbourhood.index)
        #returns the indices of all rows with the same Postal Code as the ith row
        rpc[i] = comm_inds       # assigns that list of indices as key i's values
        rpc[code] = rpc.pop(i)   # renames the key from plain number to Postal Code
        i+=1               # (i.e., appends the key-value pair to dictionary rpc)
    else:
        nother = nother + 1
        if nother==toronto_df.shape[0]:
            print("There are no repeated Postal Codes.")
    
### this segment then takes those repeats and combines their neighborhoods
i=0
k=0
new_df = toronto_df.copy()  #initiating new_df for manipulation within the loop
while i!=len(rpc):
    code = list(rpc.items())[i][0]  #a list of the ith repeated Postal Code
    inds = list(rpc.items())[i][1]  #a list of the indices containing the ith Postal Code
    temp_df = new_df[ new_df['Postal Code']==code ] #only looks at the rows with the ith Postal Code

    hoods = ""
    for ind in inds:
        bloatedlist = new_df['Neighborhood'][ind]
        hoods_list = unique_hoods(bloatedlist.split(", ")) #removes any repeat instances in a string
        slim = ""
        for h in np.arange(len(hoods_list)):
            slim = slim + hoods_list[h] + ", "
        slim = slim[:-2]
        
        hoods = hoods + slim + ", "  # creates the string of Neighboorhoods
    hoods = hoods[:-2]  #removes the trailing comma

    new_df['Neighborhood'][inds[0]] = hoods         # puts the newly formed string in the dataframe row
    new_df = pd.concat( [new_df[0:inds[0]+1],   #?# Why is shifting the indices by k-removed unnecessary?
                         new_df[inds[-1]+1:]
                        ]).reset_index(drop=True)   #removes the now redundant rows
    k=len(inds)-1
    i+=1
new_df = new_df.set_index('Postal Code')
new_df = new_df.reset_index().rename(columns = {'index':'Postal Code'})
print(new_df.shape)
new_df.head(10)

There are no repeated Postal Codes.
(103, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


# 2. Getting the LATITUDE AND LONGITUDE

In [16]:
!conda install -c conda-forge geopy --yes
from geopy import geocoders
from geopy.geocoders import GoogleV3
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



#### The following code "works" but got stuck in an endless loop trying to not return a bunch of Nones...

In [17]:
### loops to keep requesting for coordinates but gets stuck on
### the second Postal Code due to server issues...
#i=0
#all_latlngs = {}
#geolocator = Nominatim(user_agent='foursquare-agent47')

#g=None
#for code in new_df['Postal Code']:
#    while g==None:
#        g = geolocator.geocode('{}, Toronto, Ontario'.format(code))
#        i+=1
#    print('{}, Toronto, Ontario'.format(code))
#    all_latlngs[code] = [g.latitude, g.longitude]
#    g=None
### ...hence using the provided CSV file instead, below

## ...so here, I attach the retrieved Geospatial Coordinates to the DF

In [27]:
### grabs the geospatial coordinates of all of the Postal Codes from the provided CSV
latlngs_file = r'C:\Users\chris\OneDrive\Documents\Datasets\toronto_postalcode_coords.csv'
latlngs_df = pd.read_csv(latlngs_file)
latlngs_df = latlngs_df.set_index('Postal Code')  #is the same length of the new_df, which is promising

### attaches those geospatial coordinates to the Neighborhood dataframe
temp_df = new_df.copy()
temp_df = temp_df.set_index('Postal Code')
temp_df['Latitude'] = np.zeros( temp.shape[0] )
temp_df['Longitude'] = np.zeros( temp.shape[0] )

new_df2 = pd.DataFrame(columns=temp_df.columns)
for code in temp_df.index:
    new_df2 = new_df2.append( temp_df[temp_df.index==code], ignore_index=False)
       #add the row(s) from temp_df - that have the current Postal Code - to this new_df2
    new_df2['Latitude'].loc[code] = latlngs_df['Latitude'].loc[code]
    new_df2['Longitude'].loc[code] = latlngs_df['Longitude'].loc[code]
toronto_df = new_df2.reset_index().rename(columns = {'index':'Postal Code'})
toronto_df

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# 3. FourSquare and KMeans Clustering

In [30]:
#"oauth code"
CLIENT_ID = 'TNPJGNBMJOFGWKEXKU1KLF521U5C0FLS5CLLOZUFHFKJCE2I'
CLIENT_SECRET = 'MBLIWYORU1D5NS54C4AR2A2KI15IXQLZDC0CKOWMTXRPATQR'
VERSION = '20210110'
LIMIT=100
RADIUS=200
fs_url = 'http://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION,
    toronto_df['Latitude'][14], toronto_df['Longitude'][14], RADIUS, LIMIT)
# arbitrarily using the 15th row here just to see an example of the results structure
results = requests.get(fs_url).json()
results

{'meta': {'code': 200, 'requestId': '5ffbfc7ae1508a23af310faa'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 0,
  'suggestedBounds': {'ne': {'lat': 43.697143901800004,
    'lng': -79.31590379941446},
   'sw': {'lat': 43.693543898200005, 'lng': -79.32087360058554}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': []}]}}

In [31]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']  
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

def getNearbyVenues(names, latitudes, longitudes, radius=500): 
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)   
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)      
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)
toronto_venues = pd.DataFrame()
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                  latitudes=toronto_df['Latitude'],
                                  longitudes=toronto_df['Longitude'] )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [32]:
print(toronto_venues.shape)
toronto_venues.head(10)

(2121, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,Bella Vita Catering & Private Chef Service,43.756651,-79.331524,BBQ Joint
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
5,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
6,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
7,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
8,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
9,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center


In [37]:
#### now let's take a look at what we have...
print('There are {} unique categories...'.format(len(toronto_venues['Venue Category'].unique())))
print('...{} unique neighboorhoods...'.format(len(toronto_venues['Neighborhood'].unique())))
print('...and a massive {} unique venues, so we will ignore the venue names.'.format(len(toronto_venues['Venue'].unique())))

### applies OHE to the Venue Categories, assessed one Neighborhood at a time
toronto_ohe = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="|")
toronto_ohe['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_ohe.columns[-1]] + list(toronto_ohe.columns[:-1])
toronto_ohe = toronto_ohe[fixed_columns]

toronto_grouped = toronto_ohe.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(10)

There are 272 unique categories...
...97 unique neighboorhoods...
...and a massive 1387 unique venues, so we will ignore the venue names.


,Neighborhood,|Accessories Store,|Airport,|Airport Food Court,|Airport Gate,|Airport Lounge,|Airport Service,|Airport Terminal,|American Restaurant,|Antique Shop,...,|Vegetarian / Vegan Restaurant,|Video Game Store,|Video Store,|Vietnamese Restaurant,|Warehouse Store,|Wine Bar,|Wine Shop,|Wings Joint,|Women's Store,|Yoga Studio
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.017241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,"Business reply mail Processing Centre, South C...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
## a function that sorts the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

### creates a dataframe with the most common categories sorted
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)   #empty dataframe with just the columns...
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']    #now with filled in Neighborhood column also
for ind in np.arange(toronto_grouped.shape[0]):     #goes through the 40 observations/neighborhoods total
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    #each loop-iteration is a vertical numpy vector...which becomes each row of the resulting dataframe
neighborhoods_venues_sorted.head(5)   #creates a dataframe with the most common categories sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,|Lounge,|Breakfast Spot,|Latin American Restaurant,|Skating Rink,|Yoga Studio,|Eastern European Restaurant,|Dog Run,|Doner Restaurant,|Donut Shop,|Drugstore
1,"Alderwood, Long Branch",|Pizza Place,|Sandwich Place,|Coffee Shop,|Pub,|Dance Studio,|Pharmacy,|Gym,|Golf Course,|Greek Restaurant,|Eastern European Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",|Coffee Shop,|Bank,|Pet Store,|Fried Chicken Joint,|Shopping Mall,|Sandwich Place,|Diner,|Supermarket,|Middle Eastern Restaurant,|Sushi Restaurant
3,Bayview Village,|Japanese Restaurant,|Café,|Bank,|Chinese Restaurant,|Discount Store,|Distribution Center,|Dog Run,|Doner Restaurant,|Donut Shop,|Yoga Studio
4,"Bedford Park, Lawrence Manor East",|Sandwich Place,|Italian Restaurant,|Coffee Shop,|Greek Restaurant,|Toy / Game Store,|Liquor Store,|Comfort Food Restaurant,|Juice Bar,|Butcher,|Café


In [42]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
#toronto_grouped is the OHE dataframe of each neighborhood's venue type fingerprint
kmeans = KMeans(init="k-means++", n_clusters=kclusters, random_state=0) \
         .fit(toronto_grouped_clustering)
print("The labels of the", len(toronto_df), "neighborhoods are: ", list(kmeans.labels_) )
print("The",kclusters,"cluster centers have",toronto_grouped.shape[1],"dimensions each.")

The labels of the 103 neighborhoods are:  [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 3, 0, 2, 1, 1, 0, 1, 1, 2, 2]
The 5 cluster centers have 273 dimensions each.


## ...a large majority of the Neighborhoods belong to Cluster 1.

In [45]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
  #add the new cluster labels to the dataframe
toronto_merged = toronto_df.copy()
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
  #merges the neighborhood geospatial data with the venue data and their clusters

import math
inds=[]
for ind in toronto_merged.index:
#needed to rid any NaN rows that result from the merger above; see Markdown note below
    if math.isnan(toronto_merged['Cluster Labels'].iloc[ind])==True:
        inds.append(ind)

toronto_merged = toronto_merged.drop(inds).reset_index()
toronto_merged.head(10)

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,|Park,|Food & Drink Shop,|BBQ Joint,|Yoga Studio,|Drugstore,|Discount Store,|Distribution Center,|Dog Run,|Doner Restaurant,|Donut Shop
1,1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,|French Restaurant,|Portuguese Restaurant,|Hockey Arena,|Coffee Shop,|Yoga Studio,|Drugstore,|Discount Store,|Distribution Center,|Dog Run,|Doner Restaurant
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,|Coffee Shop,|Bakery,|Café,|Park,|Pub,|Theater,|Breakfast Spot,|Yoga Studio,|Beer Store,|Dessert Shop
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,|Clothing Store,|Accessories Store,|Coffee Shop,|Event Space,|Furniture / Home Store,|Athletics & Sports,|Boutique,|Vietnamese Restaurant,|Coworking Space,|Discount Store
4,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,|Coffee Shop,|Sushi Restaurant,|Yoga Studio,|Discount Store,|Beer Bar,|Japanese Restaurant,|Italian Restaurant,|Diner,|Distribution Center,|Café
5,5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,0.0,|Pizza Place,|Drugstore,|Diner,|Discount Store,|Distribution Center,|Dog Run,|Doner Restaurant,|Donut Shop,|Eastern European Restaurant,|Harbor / Marina
6,6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1.0,|Fast Food Restaurant,|Print Shop,|Eastern European Restaurant,|Discount Store,|Distribution Center,|Dog Run,|Doner Restaurant,|Donut Shop,|Drugstore,|Electronics Store
7,7,M3B,North York,Don Mills,43.745906,-79.352188,1.0,|Gym,|Beer Store,|Japanese Restaurant,|Coffee Shop,|Restaurant,|Athletics & Sports,|Sporting Goods Shop,|Sandwich Place,|Bike Shop,|Supermarket
8,8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1.0,|Pizza Place,|Gym / Fitness Center,|Gastropub,|Café,|Breakfast Spot,|Flea Market,|Bank,|Intersection,|Athletics & Sports,|Pharmacy
9,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0,|Coffee Shop,|Clothing Store,|Middle Eastern Restaurant,|Bubble Tea Shop,|Cosmetics Shop,|Japanese Restaurant,|Hotel,|Café,|Diner,|Lingerie Store


### Some of these rows contained NaN cluster labels, since the two dataframes joined are not the same length; and some of the Neighborhoods present in the original toronto_df dataframe are non-existent in the neighborhoods_venues_sorted dataframe.
### So, the NaN rows are removed...since the missing venue information is the most important part of it. They were NaN only due to the merger having no other choice.

In [46]:
import folium

geolocator = Nominatim(user_agent='foursquare-agent47')
toronto_geo = geolocator.geocode('Toronto, Ontario')
latitude = toronto_geo.latitude
longitude = toronto_geo.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)   #creates map
x = np.arange(kclusters)   #set color scheme for the clusters, here and below
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []   #add markers to the map, here and below

for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    cluster=int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

### As was pointed out before, an overwhelming majority of the Neighborhoods belong to a single cluster, namely Cluster 1. 
## As is, this KMeans Clustering implies that a large majority of the Neighborhoods in Toronto have very similar Venue variety to one another. A more feature-rich and/or more geographically inclusive dataset would likely reveal a much more class-balanced map.

### However, it may be the case that the 272-dimensional data is shaped non-spherically and could benefit from DBSCAN instead of KMeans.
## Non-Spherical DBSCAN Attempt

In [49]:
from sklearn.cluster import DBSCAN
db = DBSCAN(eps=0.3, min_samples=2).fit(toronto_grouped_clustering)
labels = db.labels_
print("The labels of the", len(toronto_grouped_clustering), "neighborhoods are: ", list(labels) )
core_mask = np.zeros_like(labels, dtype=bool)
core_mask[ db.core_sample_indices_] = True
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
unique_labels = set(labels)
print(unique_labels)

neighborhoods_venues_sorted_db = neighborhoods_venues_sorted.drop('Cluster Labels', axis=1)
neighborhoods_venues_sorted_db.insert(0, 'DBSCAN Clusters', db.labels_)
  #add the new cluster labels to the dataframe
toronto_merged_db = toronto_df.copy()
toronto_merged_db = toronto_merged_db.join(neighborhoods_venues_sorted_db.set_index('Neighborhood'), on='Neighborhood')
  #merges the neighborhood geospatial data with the venue data and their clusters

inds=[]
for ind in toronto_merged_db.index:
#needed to rid any NaN rows that result from the merger above; see Markdown note below
    if math.isnan(toronto_merged_db['DBSCAN Clusters'].iloc[ind])==True:
        inds.append(ind)

toronto_merged_db = toronto_merged_db.drop(inds).reset_index()
toronto_merged_db.head(10)

The labels of the 97 neighborhoods are:  [-1, -1, 0, -1, 0, 0, -1, 0, 0, -1, -1, 0, -1, 0, -1, 0, 0, -1, 0, 0, -1, -1, 0, -1, -1, 0, -1, -1, 0, 0, -1, 0, -1, -1, -1, 0, 0, -1, -1, -1, -1, 0, -1, -1, 0, -1, -1, -1, 0, 0, -1, -1, 0, -1, 0, -1, 0, -1, -1, -1, 0, -1, -1, 0, 0, 0, -1, -1, -1, 0, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, -1, -1, -1, 1, -1, 0, -1, -1, -1, -1, 1]
{0, 1, -1}


,index,Postal Code,Borough,Neighborhood,Latitude,Longitude,DBSCAN Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M3A,North York,Parkwoods,43.753259,-79.329656,-1.0,|Park,|Food & Drink Shop,|BBQ Joint,|Yoga Studio,|Drugstore,|Discount Store,|Distribution Center,|Dog Run,|Doner Restaurant,|Donut Shop
1,1,M4A,North York,Victoria Village,43.725882,-79.315572,-1.0,|French Restaurant,|Portuguese Restaurant,|Hockey Arena,|Coffee Shop,|Yoga Studio,|Drugstore,|Discount Store,|Distribution Center,|Dog Run,|Doner Restaurant
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,|Coffee Shop,|Bakery,|Café,|Park,|Pub,|Theater,|Breakfast Spot,|Yoga Studio,|Beer Store,|Dessert Shop
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,-1.0,|Clothing Store,|Accessories Store,|Coffee Shop,|Event Space,|Furniture / Home Store,|Athletics & Sports,|Boutique,|Vietnamese Restaurant,|Coworking Space,|Discount Store
4,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,|Coffee Shop,|Sushi Restaurant,|Yoga Studio,|Discount Store,|Beer Bar,|Japanese Restaurant,|Italian Restaurant,|Diner,|Distribution Center,|Café
5,5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,-1.0,|Pizza Place,|Drugstore,|Diner,|Discount Store,|Distribution Center,|Dog Run,|Doner Restaurant,|Donut Shop,|Eastern European Restaurant,|Harbor / Marina
6,6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,-1.0,|Fast Food Restaurant,|Print Shop,|Eastern European Restaurant,|Discount Store,|Distribution Center,|Dog Run,|Doner Restaurant,|Donut Shop,|Drugstore,|Electronics Store
7,7,M3B,North York,Don Mills,43.745906,-79.352188,0.0,|Gym,|Beer Store,|Japanese Restaurant,|Coffee Shop,|Restaurant,|Athletics & Sports,|Sporting Goods Shop,|Sandwich Place,|Bike Shop,|Supermarket
8,8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,-1.0,|Pizza Place,|Gym / Fitness Center,|Gastropub,|Café,|Breakfast Spot,|Flea Market,|Bank,|Intersection,|Athletics & Sports,|Pharmacy
9,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0.0,|Coffee Shop,|Clothing Store,|Middle Eastern Restaurant,|Bubble Tea Shop,|Cosmetics Shop,|Japanese Restaurant,|Hotel,|Café,|Diner,|Lingerie Store


#### In this case, quite a few of the neighborhoods are seen as outliers (i.e., the -1 clusters).

In [50]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)   #creates map
Dclusters = len(unique_labels)
x = np.arange(Dclusters)   #set color scheme for the DBSCAN clusters, here and below
ys = [i + x + (i*x)**2 for i in range(Dclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []   #add markers to the map, here and below

db_legend = {}
for lat, lon, poi, cluster in zip(toronto_merged_db['Latitude'], toronto_merged_db['Longitude'], toronto_merged_db['Neighborhood'], toronto_merged_db['DBSCAN Clusters']):
    cluster=int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    db_legend[cluster] = rainbow[cluster-1]
print(db_legend)
map_clusters

{-1: '#80ffb4', 0: '#ff0000', 1: '#8000ff'}


#### The above map is showing the red dots as Cluster 0 and the couple of purple dots as Cluster 1 (note this is not the same as KMeans' Cluster 0 and Cluster 1). The green dots represent DBSCAN's -1 Cluster, which means outliers that have no significant place in any given cluster.
#### Adjusting the DBSCAN's epsilon and sample minimum can present alternative arrangements, but this seems to be the most informative; and isn't that informative at all. If anything, it CONFIRMS the same thing that KMeans did, which is:
## A large majority of the Neighborhoods in Toronto have very similar Venue variety to one another. A more feature-rich and/or more geographically inclusive dataset would likely reveal a much more class-balanced map.